In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

PROJECT_PATH = "/Users/shawn/Documents/personal/rsi_divergence_detector"
# Load the training data
# Replace 'training_data.csv' with your actual data file or DataFrame
df = pd.read_pickle(f'{PROJECT_PATH}/data/training_data.pickle')
divergence_df = pd.read_pickle(f"{PROJECT_PATH}/data/divergence_data2.pickle")


In [14]:
# Test dataframes sliced
df_5m = df[df.timeframe == '5m']
df_5m_filter = df_5m.loc[df_5m.index > '2024-11-20 00:30:00']
df_5m_filter = df_5m_filter.loc[df_5m_filter.index < '2024-11-22 20:00:00']

df_total = df_5m_filter
df_divergence = divergence_df['15m'][-2:]


In [16]:
df_total.head(3)

,timestamp,open,high,low,close,volume,timeframe,rsi,macd,macd_signal,...,willr,cci,atr,return_1,return_5,return_10,volatility_5,volatility_10,volume_change,volume_rolling_mean
datetime,,,,,,,,,,,,,,,,,,,,,
2024-11-20 00:35:00,1732062900000,92080.35,92260.0,92050.53,92260.00,50.34613,5m,54.149277,9.696354,-7.417105,...,-37.900067,13.364110,194.759931,0.001951,0.000873,0.001121,0.001268,0.001520,-0.287841,58.752716
2024-11-20 00:40:00,1732063200000,92259.99,92320.0,92233.69,92233.70,69.42001,5m,53.172138,15.807042,-2.772276,...,-44.181714,87.280919,187.013507,-0.000285,0.001365,0.001378,0.001193,0.001510,0.378855,62.958378
2024-11-20 00:45:00,1732063500000,92233.69,92260.0,92124.51,92130.93,55.29224,5m,49.419335,12.216304,0.225440,...,-68.727907,-21.628155,183.333257,-0.001114,-0.000337,-0.001948,0.001317,0.001361,-0.203511,62.781558


In [17]:
df_divergence

,start_datetime,entry_datetime,entry_price,previous_peak_datetime,divergence,price_change,rsi_change,future_return,TP,SL,label
end_datetime,,,,,,,,,,,
2024-12-02 16:00:00,2024-12-01 08:45:00,2024-12-02 16:30:00,96599.99,2024-12-01 02:45:00,Bearish Divergence,-1032.69,-28.228378,-0.010607,96721.49658,97356.69,True
2024-12-06 20:30:00,2024-12-06 18:15:00,2024-12-06 21:00:00,101360.00,2024-12-06 12:45:00,Bearish Divergence,-1220.80,-22.719650,-0.011980,100223.25914,101898.99,True


In [18]:
divergence_df['15m']

,start_datetime,entry_datetime,entry_price,previous_peak_datetime,divergence,price_change,rsi_change,future_return,TP,SL,label
end_datetime,,,,,,,,,,,
2018-01-29 12:15:00,2018-01-29 01:45:00,2018-01-29 12:45:00,11263.00,2018-01-29 00:15:00,Bullish Divergence,38.00,7.708345,0.003386,11510.27164,11223.00,False
2018-07-05 00:45:00,2018-07-04 23:00:00,2018-07-05 01:15:00,6585.00,2018-07-04 21:30:00,Bullish Divergence,66.08,19.446279,0.010076,6611.42146,6557.93,True
2019-07-17 02:30:00,2019-07-16 19:15:00,2019-07-17 03:00:00,9340.52,2019-07-16 08:45:00,Bullish Divergence,150.63,14.444551,0.016295,9857.60324,9244.18,False
2019-07-26 03:15:00,2019-07-26 01:30:00,2019-07-26 03:45:00,9706.13,2019-07-25 12:45:00,Bullish Divergence,38.46,13.221338,0.003979,9853.74292,9666.54,False
2019-07-27 18:45:00,2019-07-27 12:45:00,2019-07-27 19:15:00,9405.00,2019-07-27 09:30:00,Bullish Divergence,-2.73,8.083943,-0.000291,9648.94924,9387.73,False
...,...,...,...,...,...,...,...,...,...,...,...
2024-11-13 18:45:00,2024-11-13 17:00:00,2024-11-13 19:15:00,90890.01,2024-11-13 06:30:00,Bearish Divergence,-2312.50,-20.575043,-0.024828,90461.36892,93139.93,True
2024-11-21 11:00:00,2024-11-21 04:45:00,2024-11-21 11:30:00,98020.00,2024-11-21 02:30:00,Bearish Divergence,-210.14,-8.999935,-0.002136,96725.12348,98384.86,True
2024-11-22 07:30:00,2024-11-21 04:45:00,2024-11-22 08:00:00,99173.70,2024-11-21 02:30:00,Bearish Divergence,-464.08,-17.555495,-0.004668,97364.83382,99419.99,True


In [23]:
from collections import Counter
c = Counter(list(divergence_df['15m'][-300:].label))
c

Counter({False: 199, True: 101})

In [37]:
test = divergence_df['15m']
test = test.loc[test.index > '2024-11-01']
test = test.loc[test.divergence == 'Bullish Divergence']
test['profit'] = test['TP'] - test['entry_price']
test['loss'] = test['entry_price'] - test['SL']

total_profit = 0
for idx, row in test.iterrows():
    profit = row['profit']
    loss = row['loss']
    position = row['label']

    if position:
        total_profit += profit
    else:
        total_profit -= loss

print(total_profit)

0


In [33]:
test

,start_datetime,entry_datetime,entry_price,previous_peak_datetime,divergence,price_change,rsi_change,future_return,TP,SL,label,profit,loss
end_datetime,,,,,,,,,,,,,
2024-11-04 15:00:00,2024-11-03 03:15:00,2024-11-04 15:30:00,68470.00,2024-11-02 16:45:00,Bullish Divergence,-20.96,5.993409,-0.000307,68776.49026,68257.57,True,306.49026,212.43
2024-11-25 19:45:00,2024-11-25 17:30:00,2024-11-25 20:15:00,94981.51,2024-11-25 11:15:00,Bullish Divergence,104.78,7.210027,0.001107,96126.71736,94626.42,False,1145.20736,355.09


In [35]:
c1 = Counter(list(test.loc[test.divergence == 'Bullish Divergence'][-100:].label))
c2 = Counter(list(test.loc[test.divergence == 'Bearish Divergence'][-100:].label))


In [38]:
divergence_df['1h']

,start_datetime,entry_datetime,entry_price,previous_peak_datetime,divergence,price_change,rsi_change,future_return,TP,SL,label
end_datetime,,,,,,,,,,,
2018-04-01 08:00:00,2018-03-26 18:00:00,2018-04-01 10:00:00,6763.84,2018-03-24 20:00:00,Bullish Divergence,-165.12,-5.059377,-0.024681,7560.27016,6690.12,False
2018-04-01 15:00:00,2018-03-26 18:00:00,2018-04-01 17:00:00,6783.70,2018-03-24 20:00:00,Bullish Divergence,401.92,28.163774,0.062362,7408.78600,6445.00,True
2018-08-08 17:00:00,2018-08-08 01:00:00,2018-08-08 19:00:00,6264.77,2018-08-07 16:00:00,Bullish Divergence,22.75,7.441808,0.003662,6566.66176,6212.67,False
2018-08-11 00:00:00,2018-08-08 01:00:00,2018-08-11 02:00:00,6101.35,2018-08-07 16:00:00,Bullish Divergence,-109.00,2.269193,-0.017928,6484.67170,6080.00,False
2019-09-07 03:00:00,2019-09-06 18:00:00,2019-09-07 05:00:00,10346.53,2019-09-06 15:00:00,Bullish Divergence,42.36,7.350369,0.004104,10532.92966,10320.87,False
...,...,...,...,...,...,...,...,...,...,...,...
2024-11-07 18:00:00,2024-11-06 20:00:00,2024-11-07 20:00:00,76530.40,2024-11-04 21:00:00,Bearish Divergence,-639.19,-11.911425,-0.008332,72943.30582,76718.99,False
2024-11-11 03:00:00,2024-11-10 18:00:00,2024-11-11 05:00:00,81440.01,2024-11-04 21:00:00,Bearish Divergence,-478.71,-7.869696,-0.005849,76112.23678,81846.71,False
2024-11-11 12:00:00,2024-11-10 18:00:00,2024-11-11 14:00:00,81835.73,2024-11-04 21:00:00,Bearish Divergence,2408.83,9.642991,0.029208,76498.17160,82471.20,False
